<a href="https://colab.research.google.com/github/daspartho/learn-pytorch/blob/main/experiment_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import torchvision
import torch

from torch import nn
from torchvision import transforms

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

try:
    from modular  import data_setup, engine
except:
    print("[INFO] Couldn't find modular scripts... downloading from GitHub.")
    !git clone https://github.com/daspartho/learn-pytorch
    !mv learn-pytorch/modular .
    !rm -rf learn-pytorch
    from modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find modular scripts... downloading from GitHub.
Cloning into 'learn-pytorch'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 115 (delta 49), reused 8 (delta 1), pack-reused 0
Receiving objects: 100% (115/115), 4.91 MiB | 12.53 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
def set_seeds(seed: int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [4]:
!python modular/get_data.py

Did not find data/pizza_steak_sushi directory, creating one...
Unzipping data...


In [6]:
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
transforms = weights.transforms()

train_dataloader, test_dataloader = data_setup.create_dataloaders(
    train_dir,
    test_dir,
    transforms,
    batch_size=32,
)

In [10]:
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

for param in model.features.parameters():
    param.requires_grad = False

set_seeds()

model.classifier = nn.Sequential(
    nn.Dropout(
        p=0.2, 
        inplace=True
        ),
    nn.Linear(
        in_features=1280, 
        out_features=3, 
        bias=True
        )
    )

summary(model=model,
        input_size = (32,3,224,224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        row_settings=["var_names"]
        )

Layer (type (var_name))                                      Input Shape               Output Shape              Param #                   Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]         [32, 3]                   --                        Partial
├─Sequential (features)                                      [32, 3, 224, 224]         [32, 1280, 7, 7]          --                        False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]         [32, 32, 112, 112]        --                        False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]         [32, 32, 112, 112]        (864)                     False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]        [32, 32, 112, 112]        (64)                      False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]        [32, 32, 112, 112]        --         